In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
from matplotlib import pyplot as plt
from medcat.cat import CAT

/home/wael/MedCAT/medcat/cat.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
import torch
print(torch.cuda.is_available())

True


In [31]:
DATA_DIR = "./models/"
! DATA_DIR="./models/"
model_pack_path = DATA_DIR + "kynoby_legacy_unsupervised_model_pack_UMLS/medcat_model_pack_db2b033efae16c1e"

In [32]:
cat = CAT.load_model_pack(model_pack_path, meta_cat_config_dict={'general': {'device': 'cuda'}})


/home/wael/miniforge3/envs/Medcat/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [7]:
# cat.config.general.workers = 16

In [6]:
# cat.config.general.asdict()

In [33]:
import spacy
spacy.prefer_gpu()
spacy.require_gpu()

True

In [34]:
cat.cdb.print_stats()

In [35]:
from huggingface_hub import login
# Prompt for the Hugging Face token
login()

In [36]:
import datasets
from datasets import load_dataset

In [37]:
dataset = load_dataset("saintkillio/noimg_letter_types")

Using the latest cached version of the dataset since saintkillio/noimg_letter_types couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/wael/.cache/huggingface/datasets/saintkillio___noimg_letter_types/default/0.0.0/bc973ed98a5b8d8784e2b69a3e221c9296cac10c (last modified on Sun Jul 14 00:12:39 2024).


In [38]:
dataset

DatasetDict({
    train: Dataset({
        features: ['doctr', 'letter_types', 'embeddings'],
        num_rows: 26843
    })
})

In [39]:
import pandas as pd

df = pd.DataFrame(dataset['train']) 

In [40]:
df

,doctr,letter_types,embeddings
0,"NHS 111 Report For Information DALE, David Bor...",Out of Hours,"[0.9047124981880188, -0.0075018382631242275, -..."
1,"NHS 111 Report For Information THOMPSON, Sharo...",Out of Hours,"[0.9867203235626221, 0.28119194507598877, -0.2..."
2,"NHS 111 Report - For Information NEAL, Sophie ...",Out of Hours,"[0.41013896465301514, 0.20581641793251038, -0...."
3,NHS 111 Report For Information FLAXMAN AVENUE ...,Out of Hours,"[0.6422052979469299, 0.1382981240749359, -0.18..."
4,"NHS 111 Report For Information GRIFFITHS, Kirs...",Out of Hours,"[0.7610103487968445, 0.05449499562382698, -0.0..."
...,...,...,...
26838,Lee eds Generall Infirmary: Discharge summary ...,Discharge Summary,"[0.47807037830352783, 0.33941736817359924, -0...."
26839,Leeds General Infirmary: Discharge summary Pat...,Discharge Summary,"[0.4675740599632263, 0.6129940152168274, -0.46..."
26840,St James's Institute of Oncology: Discharge su...,Discharge Summary,"[0.519070029258728, 0.4172437787055969, -0.493..."
26841,Leeds General Infirmary: Discharge summary Pat...,Discharge Summary,"[0.4644423723220825, 0.4527848958969116, -0.37..."


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26843 entries, 0 to 26842
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   doctr         26843 non-null  object
 1   letter_types  26843 non-null  object
 2   embeddings    26843 non-null  object
dtypes: object(3)
memory usage: 629.3+ KB


In [42]:
trial_text = df['doctr'][4201]
print(trial_text)

NHS Yorkshire NHS Trust ePR Sent date: 13/02/2024 YAS Reference: 4073333 GP Details: DR or Surgery: HAXBY RO OU JP &amp; NEWINGTON Ambulance Service Patient Details: Name: AN NDREW SMITH NHS Number: 4504279 096 DOB: 29/03/1955 00:00:00 Date ofincident: 12/02/2024 23:57:00 Time ofi incident: 12/02/2024 23:57:00 RT YAS Attendance Outcome Summary (Patient conveyed to hospital) -N No Action Required Presenting complaint history: PTI HAS BEEN OUT DRINKING SINCE 1300 HOURS THIS AFTERNOO ON. PTI WAS ABOUT TOL LEAVE THE VENUE TONIGHT WHEN HE WAS HEARD TO FALL ON THEF FLOOR. MOPS WENT INSIDE THE PUB AND FOUND PT ON THE FLOOR, EYES WERE OPEN HOWEVER PT APPEARED VACANT AND NOT RESPONDING. PT UNABLE TO RECALL EVENTS, MOPS ADVISE PT WAS 'SNORING' AND THEN CALLED 999 FOR NOC SPINE PAIN NOF PAIN ONF PALPATATION AND ABLE TOFREELY MOVE HEAD. B- NO INCREASED EFFORT OF BREATHING, RESPS 14 PERI MINUTE, SPO2 98%, ABLETO SPEAK INFULL SENTENCES, NO CHEST INJURIES, EQUAL CHEST RISE ANDFALL, CHEST AUSCUL ATION

In [43]:

doc = cat(trial_text)
print(doc.ents)

(NHS Trust, Sent, date, Reference, Surgery:, Date, incident, Attendance, Outcome, hospital, Action, Presenting complaint, history, DRINKING, LEAVE, FLOOR, FOUND, PT, FLOOR, PT, PT, UNABLE TO, RECALL, PT, SNORING, NOC, SPINE PAIN, PAIN, ABLE, MOVE HEAD, BREATHING, MINUTE, SPO2, SENTENCES, CHEST INJURIES, EQUAL, CHEST RISE, CLEAR, EQUAL, ENTRY, ABLE TO, SENTENCES, RADIAL, DENIES, PAIN, PRIOR, PAINFUL, PT, UNABLE, REMEMBER, STATES, PT, REMEMBERS, DRINKING, REMEMBERS, FLOOR, DENY, SEZIURE ACTIVITY, UNABLE TO, HEAD, PT, UNABLE, ALCOHOL, SANDWICH, STATES, COKES, information, Presented, assessment, CLEAR, COUGH, NORMAL, RETURNED, managing, NHS Trust, Plan, care, ASSESMENT, PT, FLOOR, MONITOR, ethanol) intoxication, PT, impression, information)


In [44]:
# To see all entities and corresponding meta-data
cat.get_entities(trial_text, only_cui=False)

{'entities': {2: {'pretty_name': 'Nhs Trust',
   'cui': 'C1273810',
   'type_ids': ['T093'],
   'types': [''],
   'source_value': 'NHS Trust',
   'detected_name': 'nhs~trust',
   'acc': 1.0,
   'context_similarity': 1.0,
   'start': 14,
   'end': 23,
   'icd10': [],
   'ontologies': ['CHV', 'SNOMEDCT_US'],
   'snomed': [],
   'id': 2,
   'meta_anns': {}},
  4: {'pretty_name': 'Send Transmission',
   'cui': 'C1519246',
   'type_ids': ['T169'],
   'types': [''],
   'source_value': 'Sent',
   'detected_name': 'sent',
   'acc': 1.0,
   'context_similarity': 1.0,
   'start': 28,
   'end': 32,
   'icd10': [],
   'ontologies': ['MTH', 'NCI'],
   'snomed': [],
   'id': 4,
   'meta_anns': {}},
  5: {'pretty_name': 'Date Property',
   'cui': 'C0011008',
   'type_ids': ['T079'],
   'types': [''],
   'source_value': 'date',
   'detected_name': 'date',
   'acc': 0.724058458856764,
   'context_similarity': 0.724058458856764,
   'start': 33,
   'end': 37,
   'icd10': [],
   'ontologies': ['MTH',
    

In [30]:
for ent in doc.ents:
    print(ent, " - ", ent._.cui)

date  -  410671006
Surgery  -  257556004
Ambulance  -  49122002
Service  -  224930009
Patient  -  116154003
SMITH  -  308323009
Number  -  410680006
Date  -  410671006
Time  -  417929005
Outcome  -  385676005
Patient  -  116154003
hospital  -  22232009
Action  -  129264002
Presenting complaint  -  33962009
history  -  392521001
DRINKING  -  30953006
HOURS  -  258702006
AFTERNOO  -  422133006
LEAVE  -  90737002
FLOOR  -  709280007
FLOOR  -  709280007
VACANT  -  422768004
UNABLE  -  371151008
EVENTS  -  272379006
SNORING  -  72863001
SPINE PAIN  -  48926009
PAIN  -  22253000
PALPATATION  -  80313002
MOVE HEAD  -  284164003
INCREASED  -  35105006
BREATHING  -  719983008
MINUTE  -  356624006
SPEAK  -  286370002
CHEST INJURIES  -  262525000
EQUAL  -  276136004
CHEST  -  51185008
CHEST  -  51185008
CLEAR  -  263707001
EQUAL  -  276136004
AIR ENTRY  -  58840004
ABLE TO SPEAK  -  286368006
STRONG  -  260404005
RADIAL  -  261173001
SINUS  -  419351001
DENIES  -  441889009
CHEST PAIN  -  2985700

In [40]:
for ent in doc.ents:
    print(ent, " - ", cat.cdb.cui2type_ids.get(ent._.cui))

Report  -  {'90170645'}
WOOD  -  {'75168589'}
Born  -  {'2680757'}
Gender  -  {'2680757'}
Female  -  {'67667581'}
Local  -  {'7882689'}
Patient  -  {'31601201'}
Home  -  {'75168589'}
Phone  -  {'32816260'}
Forest  -  {'75168589'}
Group  -  {'43039974'}
Young  -  {'7882689'}
Surgery  -  {'7882689'}
Forest  -  {'75168589'}
Phone  -  {'32816260'}
Home  -  {'75168589'}
Address  -  {'2680757'}
Emergency  -  {'7882689'}
Phone  -  {'32816260'}
Village  -  {'75168589'}
Patient  -  {'31601201'}
Reported  -  {'7882689'}
Condition  -  {'43039974'}
Disposition  -  {'46506674'}
contact  -  {'7882689'}
local  -  {'7882689'}
service  -  {'7882689'}
hours  -  {'7882689'}
service  -  {'7882689'}
referral made  -  {'3061879'}
Injury  -  {'9090192'}
illness  -  {'67667581'}
health  -  {'7882689'}
problem  -  {'67667581'}
Illness  -  {'67667581'}
Warm to touch  -  {'67667581'}
Wound  -  {'33782986'}
problem  -  {'67667581'}
Pain  -  {'67667581'}
severe  -  {'7882689'}
Wound  -  {'33782986'}
everyday  -  {

In [41]:
# We can also show the entities in a nicer way using displacy form spaCy
from spacy import displacy
displacy.render(doc, style='ent', jupyter=True)

# Unsupervised Training


In [25]:
# The first step that we want to do when using MedCAT is unsupervised training.
# You can find a full explanation of this process in the paper:
# https://doi.org/10.1016/j.artmed.2021.102083

# Print statistics on the CDB before training (note that if you are using the 
#medmen cdb it will already have some training)

cat.cdb.print_stats()

# Run the annotation procedure over all the documents we have,
#given that we have a large number of documents this can take quite some time.
print()
print(f"There are {len(df['doctr'])} documents to train...")

cat.train(df.doctr.values, progress_print=100)

# Now print statistics on the CDB after training
cat.cdb.print_stats()


There are 26843 documents to train...


KeyboardInterrupt: 

In [26]:
cat.cdb.print_stats()

In [24]:
from itertools import islice, chain
from tqdm import tqdm

# Ensure the device is set to GPU
# cat.cdb.device = 'cuda'

def batch_iterator(data, batch_size=50):
    it = iter(data)
    for first in it:
        yield list(islice(chain([first], it), batch_size - 1))

# Print stats before training
cat.cdb.print_stats()

# Determine the number of documents
num_docs = len(df['doctr'])

# Add progress bar using tqdm
batch_count = num_docs // 50 + 1  # Calculate the number of batches
for batch in tqdm(batch_iterator(df.doctr.values, batch_size=100), total=batch_count):
    cat.train(batch, progress_print=10)  # more frequent progress print

# Print stats after training
cat.cdb.print_stats()

  0%|          | 0/537 [00:00<?, ?it/s]

 51%|█████     | 272/537 [42:10<41:05,  9.30s/it]  


In [25]:
cat.cdb.make_stats()

{'Number of concepts': 354448,
 'Number of names': 2049216,
 'Number of concepts that received training': 33587,
 'Number of seen training examples in total': 23435734,
 'Average training examples per concept': 697.7620507934618}

In [ ]:
{'Number of concepts': 354448,
 'Number of names': 2049216,
 'Number of concepts that received training': 29674,
 'Number of seen training examples in total': 20585988,
 'Average training examples per concept': 693.7382220125362}
{'Number of concepts': 354448,
 'Number of names': 2049216,
 'Number of concepts that received training': 33587,
 'Number of seen training examples in total': 23435734,
 'Average training examples per concept': 697.7620507934618}
INFO:medcat.cdb:{
  "Number of concepts": 354448,
  "Number of names": 2049216,
  "Number of concepts that received training": 33587,
  "Number of seen training examples in total": 26285480,
  "Average training examples per concept": 782.6087474320422
}

In [50]:
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

try:
    logger.info("Starting training process...")
    
    # Training parameters
    batch_size = 100  # Adjust based on your system's capacity
    total_docs = len(df['doctr'])
    logger.info(f"Total documents to train: {total_docs}")
    
    # Iterate over the data in batches
    for i in range(0, total_docs, batch_size):
        batch_end = min(i + batch_size, total_docs)
        batch_data = df['doctr'].values[i:batch_end]
        
        logger.info(f"Training on batch {i // batch_size + 1} / {total_docs // batch_size + 1}")
        
        try:
            cat.train(batch_data, progress_print=100)
        except Exception as e:
            logger.error(f"Error training on batch {i // batch_size + 1}: {e}")
            continue
        
    logger.info("Training process completed successfully.")
except Exception as e:
    logger.error(f"Training process failed: {e}")


INFO:__main__:Starting training process...
INFO:__main__:Total documents to train: 26843
INFO:__main__:Training on batch 1 / 269
/home/wael/miniforge3/envs/Medcat/lib/python3.10/site-packages/cupy/cuda/compiler.py:233: PerformanceWarning: Jitify is performing a one-time only warm-up to populate the persistent cache, this may take a few seconds and will be improved in a future release...
  jitify._init_module()
INFO:medcat.cat:DONE: 100
INFO:__main__:Training on batch 2 / 269
INFO:medcat.cat:DONE: 100
INFO:__main__:Training on batch 3 / 269
INFO:medcat.cat:DONE: 100
INFO:__main__:Training on batch 4 / 269
INFO:medcat.cat:DONE: 100
INFO:__main__:Training on batch 5 / 269
INFO:medcat.cat:DONE: 100
INFO:__main__:Training on batch 6 / 269
INFO:medcat.cat:DONE: 100
INFO:__main__:Training on batch 7 / 269
INFO:medcat.cat:DONE: 100
INFO:__main__:Training on batch 8 / 269
INFO:medcat.cat:DONE: 100
INFO:__main__:Training on batch 9 / 269
INFO:medcat.cat:DONE: 100
INFO:__main__:Training on batch 

In [51]:
cat.cdb.print_stats()

INFO:medcat.cdb:{
  "Number of concepts": 4551756,
  "Number of names": 7735467,
  "Number of concepts that received training": 66765,
  "Number of seen training examples in total": 45060494,
  "Average training examples per concept": 674.9119149254849
}


In [52]:
model_pack_name = cat.create_model_pack(DATA_DIR + "kynoby_legacy_unsupervised_model_pack_UMLS")

INFO:medcat.cdb:Recalculating hash for CDB, this may take a while
INFO:medcat.cdb:Found new CDB hash: 0bfd358030df45a7
INFO:medcat.cat:Saving model pack with CDB in dill format
INFO:medcat.utils.saving.serializer:Dumping CDB to ./models/kynoby_legacy_unsupervised_model_pack_UMLS/medcat_model_pack_db2b033efae16c1e/cdb.dat
INFO:medcat.cat:{
  "Model ID": "db2b033efae16c1e",
  "Last Modified On": "14 July 2024",
  "History (from least to most recent)": [
    "0d4ccc7b9ae1ecd2",
    "17d2e290b00f11e8",
    "d0b9d8d803ef3ab0",
    "5caf9b634cbe23c1",
    "4e64b0023b1491f6",
    "62ebbdf175e4eedd",
    "198b35e8c9cd0283",
    "e0093e165b7f47e5",
    "d40143b06abf062e",
    "78605ed0f6993b4d",
    "9d9166a6561f54f1",
    "da7a3b091a29205e",
    "0f0eb8bd4a50507e",
    "4e8dcdf105b848f0",
    "60a60a4f5edcf21f",
    "d5da4386afa42c69",
    "b370b88440073ba6",
    "5b6457e57bc2bac6",
    "e6a3660d3ba19f52",
    "3760d588371755d0"
  ],
  "Description": "No description",
  "Source Ontology": null